In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import time
import numpy as np
from loguru import logger
import warnings
warnings.filterwarnings("ignore")


csv_eth_all = '../../Data/rq23/eth_alltokens.csv'
csv_eth_empty = '../../Data/rq23/eth_emptytokens.csv'
# csv_eth_all_liq = '../../Data/rq23/eth_all_liqui.csv'
csv_final = '../../Data/rq23/eth_final.csv'

chunk_size = 100000
chunks = []
# for chunk in pd.read_csv(csv_eth_all_liq, chunksize=chunk_size):
#     chunks.append(chunk)
# df_eth_all_liq = pd.concat(chunks, axis=0)
# df_eth_all_liq['timestamp'] = pd.to_datetime(df_eth_all_liq['timestamp'])
df_eth_all = pd.read_csv(csv_eth_all)
df_final = pd.read_csv(csv_final)
df_eth_empty = pd.read_csv(csv_eth_empty)

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#Not verified
# print(df_eth_all.head())
# print(df_eth_empty.head())
# print(df_eth_all_liq.head())
print('Done data loading')

In [44]:
def run_feature(days,csv_link):
        age = [] #
        is_alive = [] #   
        owner_dep = [] #
        owner_with = [] #
        owner_buy = [] # 
        owner_sell = [] #
        other_dep = [] #
        other_with = [] #
        other_buy = [] #
        other_sell = [] #
        user_count = [] #
        user_count_highest_day = [] #
        user_count_lowest_day = [] #
        user_count_first_day = [] #
        user_count_last_day = [] #
        owner_investment = [] #
        owner_return = [] #
        owner_realized = [] #
        owner_unrealized = [] #
        owner_totalprofit = [] #
        average_liquidity_affect = [] #
        min_liq_affect = [] #
        max_liq_affect = [] #
        total_volume = [] #
        highest_volume = [] #
        lowest_volume = [] #
        first_day_volume = [] #
        last_day_volume = [] #
        pool_size_first = [] #
        pool_size_last = [] #
        pool_size_highest = [] #
        pool_size_lowest = [] #

        error_tokens = []

        df_alleth_temp = df_final.copy()       
        for outer_index, outer_row in df_alleth_temp.iterrows():
                try: 
                        id = outer_row['id']
                        pool_address = outer_row['pool_address']
                        owner = outer_row['token_owner']
                        unverfied_token = outer_row['token_address']
                        verfied_token = outer_row['token_paired_address']
                        print(pool_address)

                        #DF SETTING
                        if days != 60:
                                df_tokenliq = shortrange_df(pool_address, days)
                        else: 
                                df_tokenliq = df_eth_all_liq[df_eth_all_liq['contract_address'] == pool_address]
                        df_tokenliq = df_tokenliq.sort_values(by='timestamp') #reorder by timestamp
                        df_tokenliq['date'] = df_tokenliq['timestamp'].dt.date
                        df_tokenliq_byowner = df_tokenliq[df_tokenliq['sender_address'] == owner]
                        df_tokenliq_notowner = df_tokenliq[df_tokenliq['sender_address'] != owner]
                        df_verifiedtoken = df_tokenliq[df_tokenliq['token_address'] == verfied_token]

                        #TOKEN AGE
                        min_timestamp = pd.to_datetime(df_tokenliq['timestamp'].min())
                        max_timestamp = pd.to_datetime(df_tokenliq['timestamp'].max())
                        token_age = (max_timestamp - min_timestamp).days + 1
                        alive = token_age >= days

                        #ACTION COUNT
                        if df_tokenliq_byowner.empty:
                                activity_owner = [0,0,0,0]
                        else:
                                category_counts = df_tokenliq_byowner['category'].value_counts()
                                activity_owner = [category_counts.get('deposit', 0),category_counts.get('withdraw', 0),category_counts.get('buy', 0),category_counts.get('sell', 0)]
                        if df_tokenliq_notowner.empty:
                                activity_other = [0,0,0,0]  
                        else:
                                other_counts = df_tokenliq_notowner['category'].value_counts()
                                activity_other = [other_counts.get('deposit', 0),other_counts.get('withdraw', 0),other_counts.get('buy', 0),other_counts.get('sell', 0)]

                        #Count users
                        user_count_total = df_tokenliq_notowner['sender_address'].nunique()
                        
                        unique_senders_per_day = df_tokenliq.groupby('date')['sender_address'].nunique()
                        if not unique_senders_per_day.empty:
                                user_counts = [unique_senders_per_day.values[0],unique_senders_per_day.values[-1],unique_senders_per_day.values.min(),unique_senders_per_day.values.max()]
                        else:
                                user_counts = [0,0,0,0]

                        #PROFIT
                        df_ownerdep = df_tokenliq_byowner[df_tokenliq_byowner['category'] == 'deposit']
                        df_ownerwith = df_tokenliq_byowner[df_tokenliq_byowner['category'] == 'withdraw']
                        df_ownerbuy = df_tokenliq_byowner[df_tokenliq_byowner['category'] == 'buy']
                        df_ownersell = df_tokenliq_byowner[df_tokenliq_byowner['category'] == 'sell']
                        df_ownerbuy_verified = df_ownerbuy[df_ownerbuy['token_address'] == verfied_token]
                        df_ownersell_verified = df_ownersell[df_ownersell['token_address'] == verfied_token]
                        df_ownerdep_verified = df_ownerdep[df_ownerdep['token_address'] == verfied_token]
                        df_ownerwith_verified = df_ownerwith[df_ownerwith['token_address'] == verfied_token]

                        #Realized profit
                        buy_poolgain_fromowner = (df_ownerbuy_verified['amount_token'] * df_ownerbuy_verified['token_price']).sum()
                        sell_poolloss_toowner = (df_ownersell_verified['amount_token'] * df_ownersell_verified['token_price']).sum()
                        deposit_poolgain_fromowner =  (df_ownerdep_verified['amount_token'] * df_ownerdep_verified['token_price']).sum()
                        withdraw_poolloss_toowner =  (df_ownerwith_verified['amount_token'] * df_ownerwith_verified['token_price']).sum()
                        invest = buy_poolgain_fromowner + deposit_poolgain_fromowner
                        invest = invest if invest > 0 else 1
                        returnn = -(withdraw_poolloss_toowner + sell_poolloss_toowner)
                        realized_profit = -(buy_poolgain_fromowner + sell_poolloss_toowner+deposit_poolgain_fromowner+withdraw_poolloss_toowner)

                        #Unrealized profit & total profit
                        df_tokenliq_depwith = df_tokenliq[df_tokenliq['category'].isin(['deposit', 'withdraw'])]
                        owner_share = 1
                        min_index = df_tokenliq_depwith.index.min()

                        for index, row in df_tokenliq_depwith.iterrows():
                                if row['token_address'] == unverfied_token and index != min_index:
                                        if row['sender_address'] == owner:
                                                owner_part = (row['pool_balance'] - row['amount_token'])*owner_share + row['amount_token']
                                                owner_share = owner_part/row['pool_balance'] if row['pool_balance'] != 0 else 1
                                        else:
                                                owner_part = (row['pool_balance'] - row['amount_token'])*owner_share
                                                owner_share = owner_part/row['pool_balance'] if row['pool_balance'] != 0 else 0

                                if owner_share > 1: owner_share = 1
                                elif owner_share < 0: owner_share = 0
                        last_verified = df_verifiedtoken.iloc[-1]
                        unrealized_profit = (last_verified['pool_balance']*last_verified['token_price'])*owner_share
                        total_profit = realized_profit + unrealized_profit

                        #LIQUIDITY IMPACT
                        df_ownertp = df_tokenliq_byowner[(df_tokenliq_byowner['category'].isin(['sell', 'withdraw'])) & (df_tokenliq_byowner['token_address'] == verfied_token)]
                        df_ownertp['impact'] = df_ownertp.apply(lambda row: (abs(row['amount_token']) / (row['pool_balance'] - row['amount_token']))*100, axis=1)
                        liq_impact_list = df_ownertp['impact'].tolist()
                        if liq_impact_list:
                                liq_impact_output = [min(liq_impact_list), max(liq_impact_list), np.mean(liq_impact_list)]
                        else:
                                liq_impact_output = [0,0,0]

                        #VOLUME
                        vol_per_day = df_verifiedtoken.groupby('date').apply(lambda x: (abs(x['amount_token']) * x['token_price']).sum()).reset_index(name='sum_amount_token_value')
                        if not vol_per_day.empty:
                                volume = [vol_per_day.iloc[0]['sum_amount_token_value'], vol_per_day.iloc[-1]['sum_amount_token_value'], vol_per_day['sum_amount_token_value'].min(), vol_per_day['sum_amount_token_value'].max(), vol_per_day['sum_amount_token_value'].sum()]
                        else:
                                volume = [0, 0, 0, 0,0]
                        
                        #POOL SIZE
                        last_poolsize_per_day = df_verifiedtoken.groupby('date').last().reset_index()
                        last_poolsize_per_day['pool_size'] = abs(last_poolsize_per_day['amount_token']) * last_poolsize_per_day['token_price']
                        if not last_poolsize_per_day['pool_size'].empty:
                                pool_size_perday = last_poolsize_per_day['pool_size'].tolist()
                                pool_size = [pool_size_perday[0],pool_size_perday[-1],min(pool_size_perday), max(pool_size_perday)]
                        else:
                                pool_size = [0,0,0,0]
                        logger.info(f'Success: {id}')
                except Exception as e:
                        error_tokens.append(pool_address)
                        logger.error(f'Error: {id},{e}')
                        token_age = 0
                        alive = False
                        activity_owner = [0,0,0,0]
                        activity_other = [0,0,0,0]
                        user_count_total = 0
                        user_counts = [0,0,0,0]
                        invest = 0
                        returnn = 0
                        realized_profit = 0
                        unrealized_profit = 0
                        total_profit = 0
                        liq_impact_output = [0,0,0]
                        volume = [0, 0, 0, 0, 0]
                        pool_size = [0,0,0,0]
                
                #Append in column
                age.append(token_age)
                is_alive.append(alive)
                owner_dep.append(activity_owner[0])
                owner_with.append(activity_owner[1])
                owner_buy.append(activity_owner[2])
                owner_sell.append(activity_owner[3])
                other_dep.append(activity_other[0])
                other_with.append(activity_other[0])
                other_buy.append(activity_other[0])
                other_sell.append(activity_other[0])
                user_count.append(user_count_total)
                user_count_first_day.append(user_counts[0])
                user_count_last_day.append(user_counts[1])
                user_count_lowest_day.append(user_counts[2])
                user_count_highest_day.append(user_counts[3])
                owner_investment.append(invest)
                owner_return.append(returnn)
                owner_realized.append(realized_profit)
                owner_unrealized.append(unrealized_profit)
                owner_totalprofit.append(total_profit)
                min_liq_affect.append(liq_impact_output[0])
                max_liq_affect.append(liq_impact_output[1])
                average_liquidity_affect.append(liq_impact_output[2])
                first_day_volume.append(volume[0])
                last_day_volume.append(volume[1])
                lowest_volume.append(volume[2])
                highest_volume.append(volume[3])
                total_volume.append(volume[4])
                pool_size_first.append(pool_size[0])
                pool_size_last.append(pool_size[1])
                pool_size_lowest.append(2)
                pool_size_highest.append(pool_size[3])
        
        #Update temp df
        df_alleth_temp['age'] = age
        df_alleth_temp['is_alive'] = is_alive        
        df_alleth_temp['owner_dep'] = owner_dep
        df_alleth_temp['owner_with'] = owner_with
        df_alleth_temp['owner_buy'] = owner_buy
        df_alleth_temp['owner_sell'] = owner_sell
        df_alleth_temp['other_dep'] = other_dep
        df_alleth_temp['other_with'] = other_with
        df_alleth_temp['other_buy'] = other_buy
        df_alleth_temp['other_sell'] = other_sell
        df_alleth_temp['user_count'] = user_count
        df_alleth_temp['user_count_first_day'] = user_count_first_day
        df_alleth_temp['user_count_last_day'] = user_count_last_day
        df_alleth_temp['user_count_lowest_day'] = user_count_lowest_day
        df_alleth_temp['user_count_highest_day'] = user_count_highest_day
        df_alleth_temp['user_count_first_on_peak'] = df_alleth_temp['user_count_first_day']/df_alleth_temp['user_count_highest_day']
        df_alleth_temp['user_count_first_on_low'] = df_alleth_temp['user_count_first_day']/df_alleth_temp['user_count_lowest_day']
        df_alleth_temp['user_count_last_on_peak'] = df_alleth_temp['user_count_last_day']/df_alleth_temp['user_count_highest_day']
        df_alleth_temp['user_count_last_on_low'] = df_alleth_temp['user_count_last_day']/df_alleth_temp['user_count_lowest_day']
        df_alleth_temp['user_count_low_on_peak'] = df_alleth_temp['user_count_lowest_day']/df_alleth_temp['user_count_highest_day']
        df_alleth_temp['user_count_first_on_last'] = df_alleth_temp['user_count_first_day']/df_alleth_temp['user_count_last_day']
        df_alleth_temp['owner_investment'] = owner_investment
        df_alleth_temp['owner_return'] = owner_return
        df_alleth_temp['owner_realized'] = owner_realized
        df_alleth_temp['owner_unrealized'] = owner_unrealized
        df_alleth_temp['owner_totalprofit'] = owner_totalprofit
        df_alleth_temp['owner_roi'] = df_alleth_temp['owner_return']/df_alleth_temp['owner_investment']
        df_alleth_temp['owner_realizedprofit_oninvest'] = df_alleth_temp['owner_realized']/df_alleth_temp['owner_investment']
        df_alleth_temp['owner_unrealizedprofit_oninvest'] = df_alleth_temp['owner_unrealized']/df_alleth_temp['owner_investment']
        df_alleth_temp['owner_totalprofit_oninvest'] = df_alleth_temp['owner_totalprofit']/df_alleth_temp['owner_investment']
        df_alleth_temp['min_liq_affect'] = min_liq_affect
        df_alleth_temp['max_liq_affect'] = max_liq_affect
        df_alleth_temp['average_liquidity_affect'] = average_liquidity_affect
        df_alleth_temp['liqaffect_min_on_avg'] = df_alleth_temp['min_liq_affect']/df_alleth_temp['average_liquidity_affect']
        df_alleth_temp['liqaffect_max_on_avg'] = df_alleth_temp['max_liq_affect']/df_alleth_temp['average_liquidity_affect']
        df_alleth_temp['liqaffect_min_on_max'] = df_alleth_temp['min_liq_affect']/df_alleth_temp['max_liq_affect']
        df_alleth_temp['first_day_volume'] = first_day_volume
        df_alleth_temp['last_day_volume'] = last_day_volume
        df_alleth_temp['lowest_volume'] = lowest_volume
        df_alleth_temp['highest_volume'] = highest_volume
        df_alleth_temp['total_volume'] = total_volume
        df_alleth_temp['vol_first_on_last'] = df_alleth_temp['first_day_volume']/df_alleth_temp['last_day_volume']
        df_alleth_temp['vol_first_on_low'] = df_alleth_temp['first_day_volume']/df_alleth_temp['lowest_volume']
        df_alleth_temp['vol_first_on_high'] = df_alleth_temp['first_day_volume']/df_alleth_temp['highest_volume']
        df_alleth_temp['vol_last_on_low'] = df_alleth_temp['last_day_volume']/df_alleth_temp['lowest_volume']
        df_alleth_temp['vol_last_on_high'] = df_alleth_temp['last_day_volume']/df_alleth_temp['highest_volume']
        df_alleth_temp['vol_low_on_high'] = df_alleth_temp['lowest_volume']/df_alleth_temp['highest_volume']
        df_alleth_temp['pool_size_first'] = pool_size_first
        df_alleth_temp['pool_size_last'] = pool_size_last
        df_alleth_temp['pool_size_lowest'] = pool_size_lowest
        df_alleth_temp['pool_size_highest'] = pool_size_highest
        df_alleth_temp['pool_size_first_on_last'] = df_alleth_temp['pool_size_first']/df_alleth_temp['pool_size_last']
        df_alleth_temp['pool_size_low_on_high'] = df_alleth_temp['pool_size_lowest']/df_alleth_temp['pool_size_highest']
        df_alleth_temp['pool_size_first_on_low'] = df_alleth_temp['pool_size_first']/df_alleth_temp['pool_size_lowest']
        df_alleth_temp['pool_size_first_on_high'] = df_alleth_temp['pool_size_first']/df_alleth_temp['pool_size_highest']
        df_alleth_temp['pool_size_last_on_low'] = df_alleth_temp['pool_size_last']/df_alleth_temp['pool_size_lowest']
        df_alleth_temp['pool_size_last_on_high'] = df_alleth_temp['pool_size_last']/df_alleth_temp['pool_size_highest']

        df_alleth_temp.to_csv(csv_link, index=False)
        print(df_alleth_temp)

        print(error_tokens)


In [ ]:
file_paths_1_to_300 = [f"../../Data/rq23/{day}d/eth_{day}d.csv" for day in range(1, 301)]
for file_path in file_paths_1_to_300:
    # Extract the day number from the file path
    day = int(file_path.split('/')[-1].split('_')[1].replace('d.csv', ''))
    
    # Call the function with the day and corresponding file path
    run_feature(day, file_path)